In [1]:
import numpy as np
import pandas as pd
from textblob import TextBlob
import glob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
all_files = glob.glob("./dataset_cleaning/*.pkl")
all_files.sort()
test_filename= "./dataset_cleaning/dataset4Cleaned.pkl"


li = []

for filename in all_files:
    
    if(filename != test_filename and filename != test2_filename):

        df = pd.read_pickle(filename)
        li.append(df)

train = pd.concat(li, axis=0, ignore_index=True)
test = pd.read_pickle(test_filename)

print("Number of train features: {} \nNumber of test features: {}".format(len(train), len(test)))

Number of train features: 57348 
Number of test features: 194


In [3]:
datasets

[]

In [4]:
train = pd.read_pickle('../dataset_cleaning/dataset1Cleaned.pkl')
test = pd.read_pickle('../dataset_cleaning/dataset2Cleaned.pkl')

In [5]:
test.columns

Index(['id', 'Informativeness', 'text', 'positive_score', 'negative_score',
       'emotional_devergence_score'],
      dtype='object')

In [5]:
# all_files = glob.glob("./../dataset_cleaning/*.pkl")
# all_files.sort()
# datasets = []

# for filename in all_files:
#     print(filename)
#     df = pd.read_pickle(filename)
#     datasets.append(df)
    
# from sklearn.model_selection import train_test_split
# df_merged = pd.concat(datasets)
# df_merged = df_merged.sample(frac=1).reset_index(drop=True)
# train, test= train_test_split(df_merged, test_size=0.3, random_state=42)
# print("Number of train features: {} \nNumber of test features: {}".format(len(train), len(test)))

./../dataset_cleaning/dataset1Cleaned.pkl
./../dataset_cleaning/dataset2Cleaned.pkl
./../dataset_cleaning/dataset3Cleaned.pkl
./../dataset_cleaning/dataset4Cleaned.pkl
./../dataset_cleaning/dataset5Cleaned.pkl
./../dataset_cleaning/dataset6Cleaned.pkl
./../dataset_cleaning/dataset7Cleaned.pkl
Number of train features: 45829 
Number of test features: 19642


In [6]:
authors=pd.read_csv('../dataset_cleaning/tj/tweet_metadata_full.csv')
users=pd.read_csv('../dataset_cleaning/tj/twitter_user_full.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
authors = authors.drop(columns=['Unnamed: 0'])
users = users.drop(columns=['Unnamed: 0'])

In [8]:
train=pd.merge(train, authors, on='id')
test=pd.merge(test, authors, on='id')
train.drop(train.columns.difference(['id', 'Informativeness', 'text', 'author_id', 'tweet_type','positive_score', 'negative_score',
       'emotional_devergence_score' ]), 1, inplace=True)
test.drop(test.columns.difference(['id', 'Informativeness', 'text', 'author_id', 'tweet_type','positive_score', 'negative_score',
       'emotional_devergence_score' ]), 1, inplace=True)

In [9]:
users.drop(columns=['created_at', 'lang', 'name', 'screen_name', 'location','access'], inplace=True)
users.columns=['author_id', 'has_description', 'bio_has_url', 'followers_count', 'friends_count',
       'favourites_count', 'listed_count', 'statuses_count', 'protected',
       'verified', 'default_profile', 'default_profile_image']

In [10]:
train=pd.merge(train, users, on='author_id')
test=pd.merge(test, users, on='author_id')

In [11]:
train.head()

,id,Informativeness,text,positive_score,negative_score,emotional_devergence_score,author_id,tweet_type,has_description,bio_has_url,followers_count,friends_count,favourites_count,listed_count,statuses_count,protected,verified,default_profile,default_profile_image
0,910171633578586113,1,Hurricane Irma shuts down #LGBT businesses in ...,1,-1,0.2,99041149,tweet,South Florida Gay News reports on our lives wi...,https://t.co/I92LcgAwrP,5715,5266,1930,269.0,36189,False,False,False,False
1,505040978776834048,1,"#Earthquake M 1.8, Northern California http://...",1,-1,0.2,542201905,tweet,Independent syndicated #Earthquake #quake #Inf...,http://t.co/gBmygFEo8N,981,59,0,35.0,45224,False,False,False,False
2,246585296437776384,0,"#Earthquake M 1.5, Southern California http://...",1,-1,0.2,542201905,tweet,Independent syndicated #Earthquake #quake #Inf...,http://t.co/gBmygFEo8N,981,59,0,35.0,45224,False,False,False,False
3,384891022033313793,1,"#Earthquake M 1.7, Northern California http://...",1,-1,0.2,542201905,tweet,Independent syndicated #Earthquake #quake #Inf...,http://t.co/gBmygFEo8N,981,59,0,35.0,45224,False,False,False,False
4,384077389871206400,1,"#Earthquake M 2.5, Central California http://t...",1,-1,0.2,542201905,tweet,Independent syndicated #Earthquake #quake #Inf...,http://t.co/gBmygFEo8N,981,59,0,35.0,45224,False,False,False,False


In [12]:

def text_processing(tweet):
    
    tweet= tweet.lower()
    
    #Removing hyperlinks from the tweet
    tweet_no_links=re.sub(r'http\S+', '', tweet)
    
    #Generating the list of words in the tweet (hashtags and other punctuations removed)
    def form_sentence(tweet):
        tweet_blob = TextBlob(tweet)
        return ' '.join(tweet_blob.words)
    new_tweet = form_sentence(tweet_no_links)
    
    #Removing stopwords and words with unusual symbols
    def no_user_alpha(tweet):
        tweet_list = [ele for ele in tweet.split() if ele != 'user']
        clean_tokens = [t for t in tweet_list if re.match(r'[^\W\d]*$', t)]
        clean_s = ' '.join(clean_tokens)
        clean_mess = [word for word in clean_s.split() if word not in stopwords.words('english')]
        return clean_mess
    no_punc_tweet = no_user_alpha(new_tweet)
    
    #Normalizing the words in tweets 
    def normalization(tweet_list):
        lem = WordNetLemmatizer()
        normalized_tweet = []
        for word in tweet_list:
            normalized_text = lem.lemmatize(word,'v')
            normalized_tweet.append(normalized_text)
        return " ".join(normalized_tweet)
    
    
    return normalization(no_punc_tweet)

In [13]:
def count_hashtags(text):
    return text.count('#')

In [14]:
def link_present(text):
    return int('http' in text) 

In [15]:
import emoji
def text_has_emoji(text):
    return any(map(text[:-50].__contains__, [':)',':(',':P',':c',':<','c:','<:',':L',':l','^_^','^.^','>_<','>.<','>_>','<_<','>.>','<.<','-.-','-_-','o_o','o.o','._.','owo','OwO',';_;','>:)',':]',':}','>:(','>:|','-.^','-_^','8)','B)','<3','xD',':3','x3','\o','\o/',';_;','OwO','uwu','O:)',':#',':*']))

In [16]:
def has_instructional_words(text):
    return any(map(text.__contains__, ["text", 'call', 'donate']))

In [17]:
def has_phone_number(text):
    return bool(re.search('\\d{3}-\\d{3}-\\d{4}', text))

In [18]:
def text_has_slang(text):
    for i in text[:-50].split(' '):
        for j in ['afaik','afk','asl','atm','atw','ayy','bae','bb','bbiab','bbl','bbs','bc','bf','bff','bork','brb','btw','cba','convo','cp','cya','cya','dank','dc','dem','dw','e2e','fml','FOMO','FTFY','ftl','ftw','fwiw','fyi','g2g','g4u','gf','gg','goml','gr8','gratz','gtfo','guiz','hbu','hru','ianadb','ianalb','ianap','idc','idgaf','idk','iirc','ik','ikr','ily','inb4','irl','jfc','jk','js','k','kappa','kek','kms','kthx','l8r','leet','lmao','lmk','lol','LPT','lrl','lrn2','m8','maga','mfw','mrw','nerf','ngl','nm','nmu','noob','nvm','ofc','omf','omg','omw','ooc','op','OP','orly','pepe','pleb','pleb','plz','pron','pwned','REEEEEE','rekt','rickrol','rip','rly','rms','rofl','rotflol','rtfm','rude','shank','smd','smh','soz','swag','tbf','tbh','tbt','TIFU','tf','tfw','thx','tide','TIL','tl;dr','tmw','tolo','topkek','ty','uwotm8','w00t','wb','wot','wtb','wtf','wtg','wts','wuu2','yarly','ymmv','yolo','yw']:
            if(i==j):
                return True
    return False

In [19]:
def is_RT(text):
    for i in text[:-50].split(' '):
        if(i == 'RT'):
            return True
    return False

In [20]:
def has_profanity(text):
    for i in text[:-50].split(' '):
        for j in ['acrotomophilia','anal','anilingus','anus','arsehole','ass','asshole','assmunch','auto erotic','autoerotic','babeland','baby batter','ball gag','ball gravy','ball kicking','ball licking','ball sack','ball sucking','bangbros','bareback','barely legal','barenaked','bastardo','bastinado','bbw','bdsm','beaver cleaver','beaver lips','bestiality','bi curious','big black','big breasts','big knockers','big tits','bimbos','birdlock','bitch','black cock','blonde action','blonde on blonde action','blow j','blow your l','blue waffle','blumpkin','bollocks','bondage','boner','boob','boobs','booty call','brown showers','brunette action','bukkake','bulldyke','bullet vibe','bung hole','bunghole','busty','butt','buttcheeks','butthole','camel toe','camgirl','camslut','camwhore','carpet muncher','carpetmuncher','chocolate rosebuds','circlejerk','cleveland steamer','clit','clitoris','clover clamps','clusterfuck','cock','cocks','coprolagnia','coprophilia','cornhole','cum','cumming','cunnilingus','cunt','darkie','date rape','daterape','deep throat','deepthroat','dick','dildo','dirty pillows','dirty sanchez','dog style','doggie style','doggiestyle','doggy style','doggystyle','dolcett','domination','dominatrix','dommes','donkey punch','double dong','double penetration','dp action','eat my ass','ecchi','ejaculation','erotic','erotism','escort','ethical slut','eunuch','faggot','fecal','felch','fellatio','feltch','female squirting','femdom','figging','fingering','fisting','foot fetish','footjob','frotting','fuck','fucking','fuck buttons','fudge packer','fudgepacker','futanari','g-spot','gang bang','gay sex','genitals','giant cock','girl on','girl on top','girls gone wild','goatcx','goatse','gokkun','golden shower','goo girl','goodpoop','goregasm','grope','group sex','guro','hand job','handjob','hard core','hardcore','hentai','homoerotic','honkey','hooker','hot chick','how to kill','how to murder','huge fat','humping','incest','intercourse','jack off','jail bait','jailbait','jerk off','jigaboo','jiggaboo','jiggerboo','jizz','juggs','kike','kinbaku','kinkster','kinky','knobbing','leather restraint','leather straight jacket','lemon party','lolita','lovemaking','make me come','male squirting','masturbate','menage a trois','milf','missionary position','motherfucker','mound of venus','mr hands','muff diver','muffdiving','nambla','nawashi','negro','neonazi','nig nog','nigga','nigger','nimphomania','nipple','nipples','nsfw images','nude','nudity','nympho','nymphomania','octopussy','omorashi','one cup two girls','one guy one jar','orgasm','orgy','paedophile','panties','panty','pedobear','pedophile','pegging','penis','phone sex','piece of shit','piss pig','pissing','pisspig','playboy','pleasure chest','pole smoker','ponyplay','poof','poop chute','poopchute','porn','porno','pornography','prince albert piercing','pthc','pubes','pussy','queaf','raghead','raging boner','rape','raping','rapist','rectum','reverse cowgirl','rimjob','rimming','rosy palm','rosy palm and her 5 sisters','rusty trombone','s&m','sadism','scat','schlong','scissoring','semen','sex','sexo','sexy','shaved beaver','shaved pussy','shemale','shibari','shit','shota','shrimping','slanteye','slut','smut','snatch','snowballing','sodomize','sodomy','spic','spooge','spread legs','strap on','strapon','strappado','strip club','style doggy','suck','sucks','suicide girls','sultry women','swastika','swinger','tainted love','taste my','tea bagging','threesome','throating','tied up','tight white','tit','tits','titties','titty','tongue in a','topless','tosser','towelhead','tranny','tribadism','tub girl','tubgirl','tushy','twat','twink','twinkie','two girls one cup','undressing','upskirt','urethra play','urophilia','vagina','venus mound','vibrator','violet blue','violet wand','vorarephilia','voyeur','vulva','wank','wet dream','wetback','white power','women rapping','wrapping men','wrinkled starfish','xx','xxx','yaoi','yellow showers','yiffy','zoophilia']:
            if(i==j):
                return True
    return False


In [21]:
def one_word_sentance(text):
    return int(len(text.split(" "))<=3)

In [22]:
train["hashtag_count"]=train['text'].apply(count_hashtags)
test["hashtag_count"]=test['text'].apply(count_hashtags)

In [23]:
train["hashtag_present"] = train["hashtag_count"].apply(lambda x: np.sign(x))
test["hashtag_present"] = test["hashtag_count"].apply(lambda x: np.sign(x))

In [24]:
train["link_present"]=train['text'].apply(link_present)
test["link_present"]=test['text'].apply(link_present)

In [25]:
train["emoji_present"] = train['text'].apply(text_has_emoji)
test["emoji_present"] = test['text'].apply(text_has_emoji)

In [26]:
train["instructinal_keyword"] = train['text'].apply(has_instructional_words)
test["instructinal_keyword"] = test['text'].apply(has_instructional_words)

In [27]:
train["contains_phone_number"] = train['text'].apply(has_phone_number)
test["contains_phone_number"] = test['text'].apply(has_phone_number)

In [28]:
train["slang_present"] = train['text'].apply(text_has_slang)
test["slang_present"] = test['text'].apply(text_has_slang)

In [29]:
train["is_RT"] = train['text'].apply(is_RT)
test["is_RT"] = test['text'].apply(is_RT)

In [30]:
train["has_profanity"] = train['text'].apply(has_profanity)
test["has_profanity"] = test['text'].apply(has_profanity)

In [31]:
train["one_word_sentance"] = train['text'].apply(one_word_sentance)
test["one_word_sentance"] = test['text'].apply(one_word_sentance)

In [32]:
# train = train.apply(get_sentiment_score, axis=1)
# test = test.apply(get_sentiment_score, axis=1)

In [33]:
# train["emotional_devergence_score"]=(train["positive_score"]-train["negative_score"])/10.0
# test["emotional_devergence_score"]=(test["positive_score"]-test["negative_score"])/10.0

In [34]:
test.head()

,id,Informativeness,text,positive_score,negative_score,emotional_devergence_score,author_id,tweet_type,has_description,bio_has_url,...,hashtag_count,hashtag_present,link_present,emoji_present,instructinal_keyword,contains_phone_number,slang_present,is_RT,has_profanity,one_word_sentance
0,207704417108103169,1,DTN World News: Old woman polled out alive in ...,1,-1,0.2,137840213,tweet,Comprehensive Daily News on The World of Today...,http://defense-technologynews.blogspot.com/,...,0,0,1,False,False,False,False,False,False,0
1,959839199389036546,0,This is horrifying that all over the world the...,2,-4,0.6,462478512,quote,#IFB 💯public school teacher for 20+ yrs. Proud...,NaN,...,0,0,1,False,False,False,False,False,False,0
2,850353395818954752,0,"#prayforstockholm Please, just please, dont us...",3,-4,0.7,1149393234,tweet,NaN,NaN,...,1,1,0,False,False,False,False,False,False,0
3,383510892027330560,0,Honestly I lost the count of fake degree holde...,1,-2,0.3,572666480,tweet,Die hard worker of #MQM. Love #AltafHussain,http://t.co/hDJCrkJilk,...,2,1,0,False,False,False,False,False,False,0
4,910135800750268416,1,https://t.co/IT1aCXmNFQ : HHS Offers Special M...,1,-1,0.2,164865134,tweet,Putting the Social in Supply Chain. The gather...,https://t.co/EjRpsCiQJi,...,0,0,1,False,False,False,False,False,False,0


In [35]:
['id',  'author_id', 'tweet_type','has_description', 'bio_has_url','protected', 'default_profile','default_profile_image']

['id',
 'author_id',
 'tweet_type',
 'has_description',
 'bio_has_url',
 'protected',
 'default_profile',
 'default_profile_image']

In [36]:
train= train.drop(columns=['id',  'author_id', 'tweet_type','has_description', 'bio_has_url','protected', 'default_profile','default_profile_image'])
test= test.drop(columns=['id',  'author_id', 'tweet_type','has_description', 'bio_has_url','protected', 'default_profile','default_profile_image'])

In [37]:
from tqdm import tqdm
tqdm.pandas()

train['text']=train['text'].progress_apply(text_processing)
test['text']=test['text'].progress_apply(text_processing)


/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:701: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 15971/15971 [01:30<00:00, 176.76it/s]


In [39]:

y_train = np.asarray(train.Informativeness)

In [40]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [41]:
texts=train.text
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 37909 unique tokens.


In [42]:
X_train = pad_sequences(sequences_train)
y_train = np.asarray(train.Informativeness)
print('Shape of X train :', X_train.shape)
print('Shape of label train :', y_train.shape)

Shape of X train : (37203, 464)
Shape of label train : (37203,)


In [43]:
max(train.text.apply(lambda x: len(x.split(' '))))


464

In [47]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('../word2vec_data/GoogleNews-vectors-negative300.bin', binary=True)

EMBEDDING_DIM=300
vocabulary_size=len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)



from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [48]:
embedding_matrix.shape

(37910, 300)

In [49]:
embedding_matrix.shape

(37910, 300)

In [50]:
train.shape

(37203, 21)

In [53]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = X_train.shape[1]

filter_sizes = [3,4,5]
num_filters = 150
drop = 0.5



inputs = Input(shape=(sequence_length,))

#CHANGE THIS!
meta_input = Input(shape=(19,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)


conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
merged_tensor2 = concatenate([dropout, meta_input])

output = Dense(units=2, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(merged_tensor2)



model = Model([inputs , meta_input], output)

In [54]:
y_train = y_train.reshape(len(y_train), 1)

y_train_extra = np.abs(y_train-1)

X_train_meta = train.to_numpy()[:, 2:].astype('float32')

In [57]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
callbacks = [EarlyStopping(monitor='loss')]
model.fit([X_train, X_train_meta], np.append(y_train, y_train_extra, axis=1), batch_size=1000, epochs=10, verbose=1,
         callbacks=callbacks)  


Epoch 1/10
38/38 [==============================] - 896s 24s/step - loss: 5495.5020 - acc: 0.6644
Epoch 2/10
38/38 [==============================] - 872s 23s/step - loss: 2296.1868 - acc: 0.6178
Epoch 3/10
38/38 [==============================] - 800s 21s/step - loss: 295.1976 - acc: 0.6324
Epoch 4/10
38/38 [==============================] - 897s 24s/step - loss: 61.7619 - acc: 0.7471
Epoch 5/10
38/38 [==============================] - 845s 22s/step - loss: 39.3861 - acc: 0.7699
Epoch 6/10
38/38 [==============================] - 777s 20s/step - loss: 74.8804 - acc: 0.7641


In [58]:
test.columns

Index(['Informativeness', 'text', 'positive_score', 'negative_score',
       'emotional_devergence_score', 'followers_count', 'friends_count',
       'favourites_count', 'listed_count', 'statuses_count', 'verified',
       'hashtag_count', 'hashtag_present', 'link_present', 'emoji_present',
       'instructinal_keyword', 'contains_phone_number', 'slang_present',
       'is_RT', 'has_profanity', 'one_word_sentance'],
      dtype='object')

In [59]:
train.columns

Index(['Informativeness', 'text', 'positive_score', 'negative_score',
       'emotional_devergence_score', 'followers_count', 'friends_count',
       'favourites_count', 'listed_count', 'statuses_count', 'verified',
       'hashtag_count', 'hashtag_present', 'link_present', 'emoji_present',
       'instructinal_keyword', 'contains_phone_number', 'slang_present',
       'is_RT', 'has_profanity', 'one_word_sentance'],
      dtype='object')

In [60]:
sequences_test=tokenizer.texts_to_sequences(test.text)
X_test = pad_sequences(sequences_test,maxlen=X_train.shape[1])
X_test_meta = test.to_numpy()[:, 2:].astype('float32')
y_pred=model.predict([X_test, X_test_meta])

In [61]:
y_pred_bool =(y_pred > 0.5).astype("int32")

In [62]:
y_true = np.asarray(test.Informativeness).reshape(len(test),1)

In [63]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
print("Acc: {}, f1: {}, roc: {}".format(accuracy_score(y_true, y_pred_bool[:,0]),f1_score(y_true, y_pred_bool[:,0]),roc_auc_score(y_true, y_pred_bool[:,0])))


Acc: 0.7156721557823555, f1: 0.7841011743450768, roc: 0.6945941640545097
